https://pyomo.readthedocs.io/en/stable/ <br>
https://pyomo.readthedocs.io/en/stable/working_models.html

In [1]:
import pyomo.environ as pyo
import pandas as pd

## Eingabedaten

In [2]:
input_data = pd.read_excel("Inputdata.xlsx", index_col="Land")

input_data

input_data.to_dict()

input_data["H2 Herstellungspreis"].to_dict()

## Solver

In [3]:
solver = pyo.SolverFactory('gurobi') 

## Model

In [4]:
model_2 = pyo.ConcreteModel()

## Länder als itterable

input_data.index

In [5]:
model_2.country = input_data.index

## Parameter

In [6]:
# Parameter in Model importieren
model_2.h2_price =  pyo.Param(model_2.country, initialize=input_data["H2 Herstellungspreis"].to_dict())
model_2.h2_pipeline =  pyo.Param(model_2.country, initialize=input_data["H2 Pipeline Preis"].to_dict())
model_2.h2_ship =  pyo.Param(model_2.country, initialize=input_data["H2 Schiff Preis"].to_dict())

model_2.nh3_price =  pyo.Param(model_2.country, initialize=input_data["NH3 Herstellungspreis"].to_dict())
model_2.nh3_pipeline =  pyo.Param(model_2.country, initialize=input_data["NH3 Pipeline Preis"].to_dict())
model_2.nh3_ship =  pyo.Param(model_2.country, initialize=input_data["NH3 Schiff Preis"].to_dict())

model_2.ch3oh_price =  pyo.Param(model_2.country, initialize=input_data["CH3OH Herstellungspreis"].to_dict())
model_2.ch3oh_pipeline =  pyo.Param(model_2.country, initialize=input_data["CH3OH Pipeline Preis"].to_dict())
model_2.ch3oh_ship =  pyo.Param(model_2.country, initialize=input_data["CH3OH Schiff Preis"].to_dict())

model_2.h2_price.pprint()

## Variablen

In [7]:
# Variablen definieren
model_2.h2_amount_ship = pyo.Var(model_2.country, domain = pyo.NonNegativeReals)
model_2.h2_amount_pipeline = pyo.Var(model_2.country, domain = pyo.NonNegativeReals)

model_2.nh3_amount_ship = pyo.Var(model_2.country, domain = pyo.NonNegativeReals)
model_2.nh3_amount_pipeline = pyo.Var(model_2.country, domain = pyo.NonNegativeReals)

model_2.ch3oh_amount_ship = pyo.Var(model_2.country, domain = pyo.NonNegativeReals)
model_2.ch3oh_amount_pipeline = pyo.Var(model_2.country, domain = pyo.NonNegativeReals)

## Zielfunktion

In [8]:
# Zielfunktion
obj_expr = sum(model_2.h2_pipeline[i] * model_2.h2_amount_pipeline[i] for i in model_2.country)
model_2.obj = pyo.Objective(sense=pyo.minimize, expr=obj_expr)

# müssen noch gemacht werden

## Nebenbedingungen

In [9]:
# Leere Liste von NB
model_2.constraints = pyo.ConstraintList()

# NB definieren
con_test_lhs = sum(model_2.h2_amount_pipeline[i] for i in model_2.country)
con_test_rhs = 10

con_demand_lhs = sum(model_2.h2_amount_pipeline[i] for i in model_2.country)
con_demand_rhs = 20

# NB hinzufügen
model_2.constraints.add(con_test_lhs >= con_test_rhs)
model_2.constraints.add(con_demand_lhs <= con_demand_rhs)

model_2.pprint()

In [10]:
# Create a model instance and optimize
instance = model_2.create_instance()
results = solver.solve(instance)
instance.display()

Model unknown

  Variables:
    h2_amount_ship : Size=4, Index=h2_amount_ship_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  None :  None : False :  True : NonNegativeReals
          B :     0 :  None :  None : False :  True : NonNegativeReals
          C :     0 :  None :  None : False :  True : NonNegativeReals
         De :     0 :  None :  None : False :  True : NonNegativeReals
    h2_amount_pipeline : Size=4, Index=h2_amount_pipeline_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  10.0 :  None : False : False : NonNegativeReals
          B :     0 :   0.0 :  None : False : False : NonNegativeReals
          C :     0 :   0.0 :  None : False : False : NonNegativeReals
         De :     0 :   0.0 :  None : False : False : NonNegativeReals
    nh3_amount_ship : Size=4, Index=nh3_amount_ship_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  None :  None : False